In [32]:
import sys
from pathlib import Path
# code_path = Path('../').resolve()
# sys.path.append(str(code_path))
import _path

In [ ]:
from firefly_optimizer import *
from plot_results import *
from transition_probability_estimation import *
from Data_synthesize import *
from rw_data_processing import *
from R0_network import R0_average_effective_contact
from scipy.optimize import leastsq
import random
import seaborn as sns
import queue
import scipy.stats as stats
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import powerlaw
import time
%load_ext autoreload
%autoreload 
plt.style.use(r"../rw_visualization.mplstyle")


In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = seaborn.color_palette()
current_palette


# 1. Illustrations of epidemiological statistics

In [35]:
# Byrne2020_Inferred_duration_of_infectious_period_of_SARS-CoV-2_rapid_scoping_review_and_analysis_of_available_evidence_for_asymptomatic_and_symptomatic_COVID-19_cases
report_asymptomatic_infectious_period = {'Zhou': [11, np.nan, np.nan], 'Hu': [9.5, 1, 21], 
                                    'Ma': [7.25, 5.91, 8.69], 'Hu': [np.nan, 4, 9], 
                                    'Li': [3.5, 3.19, 3.78], 'Tuite': [np.nan, 6, 6.5],
                                    'Davies': [7, np.nan, np.nan], 'Davies': [5, np.nan, np.nan]}
report_presymptomatic_infectious_period = {'Pan': [1, np.nan, np.nan], 'Hoehl': [1, np.nan, np.nan],
                                      'Huang': [4, 3, 5], 'Rothe': [2, 1, 3],
                                      'He': [2.3, 0.8, 3], 'Wei': [2.5, 2, 3], 
                                      'Peak': [0.8, -0.29, 1.98], 'Zhu': [1, np.nan, np.nan], 
                                      'Davies': [2.4, np.nan, np.nan], 'Tuite': [np.nan, 0.5, 1],
                                      'Ferguson': [0.5, np.nan, np.nan], 'Tindale': [2.9, 1.2, 8.2]
                                      }
report_postsymptomatic_infectious_period = {
    # Previous entries
    'Cai_et_al_1': [12.0, 6.0, 22.0],      # Median with 6-22 range
    'Cai_et_al_2': [14.0, 9.0, 19.0],      # Median with 9-19 IQR
    'Chen_et_al_1': [12.0, np.nan, np.nan], # Max
    'Cheng_et_al': [21.0, np.nan, np.nan],  # Max
    'Hu_et_al': [12.0, 12.0, 14.0],        # Median with 12-14 IQR
    'Kimball_et_al': [15.5, 14.0, 17.0],   # Median with 14-17 range
    'Kujawski_et_al': [26.0, np.nan, np.nan], # Max
    'Lee_et_al': [20.0, np.nan, np.nan],    # Max
    'Lim_et_al': [16.0, np.nan, np.nan],    # Max
    'Ling_et_al': [9.5, 2.0, 22.0],        # Median with 2-22 range
    'Liu_et_al': [11.0, 7.0, 18.0],        # Median with 7-18 range
    'Marchand_Senecal': [23.0, np.nan, np.nan], # Max
    'Pan_et_al': [10.0, 8.0, 12.0],        # Median with 8-12 range
    'Qu_et_al': [22.0, np.nan, np.nan],     # Max
    'Tan_et_al': [16.0, np.nan, np.nan],    # Max
    'Thevarajan_et_al': [7.0, np.nan, np.nan], # Max
    'Xing_et_al': [14.0, np.nan, np.nan],    # Median
    'Young_et_al': [12.5, np.nan, np.nan],  # Median
    'Yuan_et_al': [6.0, 4.0, 10.0],        # Median with 4-10 IQR
    'Zhou_et_al': [20.0, 16.0, 23.0],      # Median with 16-23 IQR
    'Chen_et_al_2': [11.0, 10.0, 12.0],    # Median with 95% CI
    'Fang_et_al_1': [15.7, 15.7-6.7, 15.7+6.7],  # Mean with SD 6.7
    'Fang_et_al_2': [22.3, 22.3-3.6, 22.3+3.6],  # Mean with SD 3.6
    'Hill_et_al': [9.0, np.nan, np.nan],    # Max
    'Le_et_al': [12.0, np.nan, np.nan],     # Max
    'Liu_et_al_2': [10.0, np.nan, np.nan],  # Max
    'Qiu_et_al': [10.0, 7.0, 22.0],        # Mean with range
    'To_et_al': [25.0, np.nan, np.nan],     # Max
    'Wu_et_al': [16.1, 16.1-6.7, 16.1+6.7], # Mean with SD 6.7
    'Tindale_et_al': [18.0, 9.0, 33.0],    # Median with range
    'Kraemer_et_al_1': [19.0, 3.0, 37.0],  # Median with range (published as Xu et al)
    'Linton_et_al': [13.0, 6.0, 41.0],     # Median with range
    'Kraemer_et_al_2': [19.25, 12.0, 24.0], # Mean with range
    'Wolfel_et_al': [4.0, 3.0, 8.0]        # Absolute with range (using middle of 3-8 days)
}

report_infectious_period = {
    # Tracking studies
    'He_et_al': [9.3, 7.8, 10.0],          # Mean with 95% CI
    'Ma_et_al': [5.0, 0.0, 24.0],          # Median with range
    'Li_et_al': [3.45, 3.19, 3.72],        # Median with 95% CI for mean
    'Tuite_et_al': [6.25, 6.0, 6.5],       # Fixed parameter (using midpoint of 6-6.5)
    'Lourenco_et_al': [3.5, 3.0, 6.0],     # Mean with 95% CI (using midpoint of 3-4 days)
    'Zhu_et_al': [12.5, 12.5-np.sqrt(11.4), 12.5+np.sqrt(11.4)],  # Mean with variance 11.4
    'Davies_et_al_1': [7.0, np.nan, np.nan], # Mean (prior)
    'Davies_et_al_2': [5.0, np.nan, np.nan], # Mean (prior)
    'Piccolomini_Zama': [20.0, np.nan, np.nan]  # Fixed
}

In [ ]:
R0 = [['Wu et al. (2019-12-31 to 2020-01-28, China)', 2.68, 2.47, 2.86], ['Shen et al. (2019-12-31 to 2020-01-28, China)', 4.71, 4.5, 4.92], ['Liu et al. (2019-12-29 to 2020-01-23, China)', 2.90, 2.36, 3.63], ['Liu et al. (2019-12-29 to 2020-01-23, China)', 2.92, 2.28, 3.67], ['Read et al. (2020-01-01 to 2020-01-22, China)', 3.11, 2.39, 4.13], ['Majumber et al. (2019-12-08 to 2020-01-26, China)', 2.55, 2, 3.1], ['Zhao et al. (2020-01-10 to 2020-01-24, China)', 2.24, 1.96, 2.55], ['Zhao et al. (2020-01-10 to 2020-01-24, China)', 3.58, 2.89, 4.39], ['Imai et al. (2019-12-01 to 2020-01-24, China)', 2.6, 1.5, 3.5], ['Riou et al. (2019-12-31 to 2020-01-28, China)', 2.2, 1.4, 3.8], ['Tang et al. (2019-12-31 to 2020-01-28, China)', 6.47, 5.71, 7.23], ['Li et al. (2019-12-01 to 2020-01-24, China)', 2.2, 1.4, 3.9], ['Zhang et al. (2019-12-08 to 2020-01-22, China)', 2.28, 2.06, 2.52], ['Du et al. (2019-12-31 to 2020-01-30, China)', 1.9, 1.47, 2.59], ['Muniz-Rodriguez et al. (2020-01-20 to 2020-02-09, China)', 3.3, 3.1, 4.2], ['Zhou et al. (2019-12-01 to 2020-01-24, China)', 2.12, 2.04, 2.18], ['Liu et al. (2019-12-08 to 2020-01-22, China)', 4.5, 4.4, 4.6], ['Liu et al. (2019-12-08 to 2020-01-22, China)', 4.4, 4.3, 4.6], ['Li et al. (2019-12-31 to 2020-01-23, China)', 2.38, 2.03, 2.77], ['Park et al. (2019-12-12 to 2020-01-22, China)', 3.1, 2.1, 5.7], ['Shao et al. (2019-12-19 to 2020-02-16, China)', 3.32, 3.25, 3.4], ['Zhou et al. (2019-12-01 to 2020-01-24, China)', 5.5, 5.3, 5.8], ['Lai et al. (2019-12-19 to 2020-02-16, China)', 2.6, 2.1, 5.1], ['Jung et al. (2019-12-31 to 2020-01-28, China)', 2.1, 2, 2.2], ['Jung et al. (2019-12-31 to 2020-01-28, China)', 3.2, 2.7, 3.7], ['Sanche et al. (2019-12-01 to 2020-01-23, China)', 6.3, 3.3, 11.3], ['Sanche et al. (2019-12-01 to 2020-01-23, China)', 4.7, 2.8, 7.6]]
R0_RW = [['Anastassopoulou et al. (2020-01-11 to 2020-02-10, China)', 2.6, 2.6, 2.6], ['Anastassopoulou et al. (2020-01-11 to 2020-02-10, China)', 2, 2, 2], ['Rajendrakumar et al. (2019-11-16 to 2020-02-10, China)', 1.47, 1.43, 1.51], ['Rajendrakumar et al. (2020-01-16 to 2020-02-29, China)', 1.89, 1.64, 2.15], ['Ahmadi et al. (2020-02-19 to 2020-03-16,  Iran​)', 1.75, 1.75, 1.75], ['Dropkin et al. (2020-01-30 to 2020-04-13, UK)', 5.81, 5.81, 5.81], ['Yuan et al. (2020-02-23 to 2020-03-09, Italy)', 3.27, 3.17, 3.38], ['Yuan et al. (2020-02-23 to 2020-03-09, France)', 6.32, 5.72, 6.99], ['Yuan et al. (2020-02-21 to 2020-03-09, Germany)', 6.07, 5.51, 6.69], ['Yuan et al. (2020-02-19 to 2020-03-09, Spain)', 5.08, 4.51, 5.74], ['Yuan et al. (2020-02-23 to 2020-03-09)', 3.1, 2.21, 4.11], ['Yuan et al. (2020-02-23 to 2020-03-09, France)', 6.56, 2.04, 12.26], ['Yuan et al. (2020-02-21 to 2020-03-09, Germany)', 4.43, 1.83, 7.92], ['Yuan et al. (2020-02-19 to 2020-03-09, Spain)', 3.95, 0, 10.19], ['Wu et al. (2019-12-01 to 2020-01-24, China)', 0.3, 0.17, 0.44], ['Russo et al. (2020-02-21 to 2020-03-08, Italy)', 4.04, 4.03, 4.05], ['Misicu et al. (2019-12-01 to 2020-01-24, China)', 1.93, 1.93, 1.93]]

min_R0 = 100
max_R0 = 0
for i in range(len(R0)):
    min_R0 = min(min(R0[i][1::]), min_R0)
    max_R0 = max(max(R0[i][1::]), max_R0)

for i in range(len(R0_RW)):
    min_R0 = min(min(R0_RW[i][1::]), min_R0)
    max_R0 = max(max(R0_RW[i][1::]), max_R0)

min_mean_R0 = 100
max_mean_R0 = 0
for i in range(len(R0)):
    min_mean_R0 = min(R0[i][1], min_mean_R0)
    max_mean_R0 = max(R0[i][1], max_mean_R0)

for i in range(len(R0_RW)):
    min_mean_R0 = min(R0_RW[i][1], min_mean_R0)
    max_mean_R0 = max(R0_RW[i][1], max_mean_R0)    

print('Mean R0 range: ', [min_mean_R0, max_mean_R0])
print('R0 CI range: ', [min_R0, max_R0])

In [ ]:
# Latent period
# Xin, H., Li, Y., Wu, P., Li, Z., Lau, E.H., Qin, Y., Wang, L., Cowling, B.J., Tsang, T.K., Li, Z.: Estimating the latent period of coronavirus disease 2019 (covid-19). Clinical Infectious Diseases 74(9), 1678–1681 (2022)
# Cheng, H.-Y., Jian, S.-W., Liu, D.-P., Ng, T.-C., Huang, W.-T., Lin, H.-H.: Contact Tracing Assessment of COVID-19 Transmission Dynamics in Taiwan and Risk at Diff erent Exposure Periods Before and After Symptom Onset. JAMA Internal Medicine 180(9), 1156 (2020). 
# Cheng's estimates are median
report_latent_period = {'Xin': [5.5, 5.1, 5.9], 'Cheng': [4.1, 0.1, 27.8]}
min_latent_period = 100
max_latent_period = 0
min_mean_latent_period = 100
max_mean_latent_period = 0
for key, value in report_latent_period.items():
    min_latent_period = min(min_latent_period, value[1])
    max_latent_period = max(max_latent_period, value[2])
    min_mean_latent_period = min(min_mean_latent_period, value[0])
    max_mean_latent_period = max(max_mean_latent_period, value[0])
print('Mean latent period range: ', [min_mean_latent_period, max_mean_latent_period])
print('Latend period CI range: ', [min_latent_period, max_latent_period], '\n')

# Incubation period
# Cheng, H.-Y., Jian, S.-W., Liu, D.-P., Ng, T.-C., Huang, W.-T., Lin, H.-H.: Contact Tracing Assessment of COVID-19 Transmission Dynamics in Taiwan and Risk at Diff erent Exposure Periods Before and After Symptom Onset. JAMA Internal Medicine 180(9), 1156 (2020). 
# Quesada, J. A., López-Pineda, A., Gil-Guillén, V. F., Arriero-Marín, J. M., Gutiérrez, F., & Carratala-Munuera, C. (2021). Incubation period of COVID-19: A systematic review and meta-analysis. Revista Clínica Española (English Edition), 221(2), 109-117.
# Park, M., Cook, A. R., Lim, J. T., Sun, Y., & Dickens, B. L. (2020). A systematic review of COVID-19 epidemiology based on current evidence. Journal of clinical medicine, 9(4), 967.
report_incubation_period = {'Cheng': [4.1, 0.4, 15.8], 'Li': [5.2, 4.1, 7.0], 'Backer':[6.5, 5.6, 7.9], 
'Linton':[5, 4.2, 6.0], 'Linton2': [5.6, 5, 6.3], 'Tian': [6.7, np.nan, np.nan], 'Song': [5, 4.3, 5.7], 
'Lauer': [5.1, 4.5, 5.8], 'Jiang': [4.9, 4.4, 5.5], 'Leung': [7.2, 6.1, 8.4],
'Quian': [6, np.nan, np.nan], 'Guan': [4, np.nan, np.nan], 'Ki': [3.9, np.nan, np.nan], 
'Xu': [4, np.nan, np.nan], 'Xu2': [8, np.nan, np.nan], 'Wu': [6, np.nan, np.nan], 'Zhang': [5.2, 1.8, 12.4]}
min_incubation_period = 100
max_incubation_period = 0
min_mean_incubation_period = 100
max_mean_incubation_period = 0
for key, value in report_incubation_period.items():
    min_incubation_period = min(min_incubation_period, value[1])
    max_incubation_period = max(max_incubation_period, value[2])
    min_mean_incubation_period = min(min_mean_incubation_period, value[0])
    max_mean_incubation_period = max(max_mean_incubation_period, value[0])

print('Mean incubation period range: ', [min_mean_incubation_period, max_mean_incubation_period])
print('Incubation period CI range: ', [min_incubation_period, max_incubation_period])


In [ ]:
# Knight, J., & Mishra, S. (2020). Estimating effective reproduction number using generation time versus serial interval, with application to COVID-19 in the Greater Toronto Area, Canada. Infectious Disease Modelling, 5, 889-896.
# Deng, Y., You, C., Liu, Y., Qin, J., & Zhou, X. H. (2021). Estimation of incubation period and generation time based on observed length-biased epidemic cohort with censoring for COVID-19 outbreak in China. Biometrics, 77(3), 929-941.
# Griffin, J., Casey, M., Collins, Á., Hunt, K., McEvoy, D., Byrne, A., ... & More, S. (2020). Rapid review of available evidence on the serial interval and generation time of COVID-19. BMJ open, 10(11), e040263.
# Generation time
def gamma_ci(alpha, beta, confidence=0.95):
    """
    Calculate confidence intervals for Gamma distribution
    using shape (alpha) and scale (beta) parameters
    """
    # Create gamma distribution
    gamma_dist = stats.gamma(a=alpha, scale=beta)
    
    # Calculate percentiles for CI
    lower_percentile = (1 - confidence) / 2
    upper_percentile = 1 - lower_percentile
    
    ci_lower = gamma_dist.ppf(lower_percentile)
    ci_upper = gamma_dist.ppf(upper_percentile)
    
    mean = alpha * beta
    
    return {
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    }

# Generation time data from the table with Gamma parameters
generation_time_data = {
    'this': {
        'alpha': 1.813,
        'beta': 2.199
    },
    'Ganyani_et_al_1': {
        'alpha': 9.140,
        'beta': 0.569
    },
    'Ganyani_et_al_2': {
        'alpha': 6.843,
        'beta': 0.577
    }
}

# Calculate results for each study
results = {}
for study, params in generation_time_data.items():
    results[study] = gamma_ci(params['alpha'], params['beta'])
    
# Print results
print("Generation Time Estimates from Gamma Distributions (95% CI):")
for study, res in results.items():
    print(f"{study}:")
    print(f"  Mean: {res['mean']:.2f} days")
    print(f"  95% CI: [{res['ci_lower']:.2f}, {res['ci_upper']:.2f}] days")

report_generation_time = {'Knight': [3.99, 0.41, 11.53], 'Ganyani': [5.2, 2.4, 9.8], 
'Ganyani2': [3.95, 1.57, 7.41], 'Deng':[2.9, 2.15, 3.86], 'Ferretti':[5, np.nan, np.nan]}
min_generation_time = 100
max_generation_time = 0
min_mean_generation_time = 100
max_mean_generation_time = 0
for key, value in report_generation_time.items():
    min_generation_time = min(min_generation_time, value[1])
    max_generation_time = max(max_generation_time, value[2])
    min_mean_generation_time = min(min_mean_generation_time, value[0])
    max_mean_generation_time = max(max_mean_generation_time, value[0])

In [ ]:
# Serial interval
# Park, M., Cook, A. R., Lim, J. T., Sun, Y., & Dickens, B. L. (2020). A systematic review of COVID-19 epidemiology based on current evidence. Journal of clinical medicine, 9(4), 967.
# Griffin, J., Casey, M., Collins, Á., Hunt, K., McEvoy, D., Byrne, A., ... & More, S. (2020). Rapid review of available evidence on the serial interval and generation time of COVID-19. BMJ open, 10(11), e040263.
report_serial_interval = {'Li': [7.5, 5.3, 19.0], 'Ki':[4.6, 3, 9], 
'Du':[3.96, 3.53, 4.39], 'Zhang':[5.1, 1.3, 11.6], 'Nishiura':[4, 3.1, 4.9],
'Nishiura2':[4.6, 3.5, 5.9], 'S. Zhao':[4.4, 2.9, 6.7], 'Wang':[5.2, 3.8, 6.8],
    'Du_Outside_Hubei': [3.96, 3.53, 4.39],
    'Ren': [5.7, 4.0, 6.8],
    'Zhang_Outside_Hubei': [5.1, 1.3, 11.6],
    'Ali': [5.1, 4.7, 5.5],
    'Xu': [5.1, 4.7, 5.5],
    'You': [4.6, np.nan, np.nan],
    'Wang_Beijing': [7.6, 6.4, 8.9],
    'Bi': [6.3, 5.2, 7.6],
    'Wu_Shenzhen': [5.9, 3.9, 9.6],
    'Qin': [6.5, 4.8, 8.2],
    'Liao': [6.5, 2.5, 17.4],
    'Wang_Tianjin': [4.8, np.nan, np.nan],
    'Wu_Zhuhai': [6.3, np.nan, np.nan],
    'Zhao_HK': [5.2, np.nan, np.nan],
    'Kwok_1': [4.77, 3.47, 6.90],
    'Kwok_2': [6.23, 4.71, 8.63],
    'Bao': [4.4, 3.3, 5.4],
    'Liu_Taiwan': [5.1, np.nan, np.nan],
    'Bae_SK': [5.2, 1.4, 9.0],
    'Mettler': [3.43, 2.62, 4.24],
    'Son': [5.54, 4.08, 7.01],
    'Ki_SK': [6.6, np.nan, np.nan],
    'Pham': [3.24, 1.38, 5.10],
    'Wong': [5.4, 4.3, 6.5],
    'Aghaali': [4.55, np.nan, np.nan],
    'Najafi': [5.71, np.nan, np.nan],
    'Prete': [3.03, 2.26, 3.73],          # Brazil
    'Lavezzo': [7.2, 5.9, 9.6],           # Italy
    'Ganyani_Singapore': [5.21, -3.35, 13.94],
    'Ganyani_Tianjin': [3.95, -4.47, 12.51],
    'He': [5.8, 4.8, 6.8],                # Multiple Asian countries
    'Nishiura_1': [4.7, 3.7, 6.6],        # Multiple Asian countries + Germany
    'Nishiura_2': [4.8, 3.8, 6.1],        # Multiple Asian countries + Germany
    'Tindale_Singapore': [4.17, 2.44, 5.89],
    'Tindale_Tianjin': [4.31, 2.91, 5.72],
    'Wu_Multiple': [7.0, 5.8, 8.1]         # Multiple countries
    }
min_serial_interval = 100
max_serial_interval = 0
min_mean_serial_interval = 100
max_mean_serial_interval = 0
for key, value in report_serial_interval.items():
    min_serial_interval = min(min_serial_interval, value[1])
    max_serial_interval = max(max_serial_interval, value[2])
    min_mean_serial_interval = min(min_mean_serial_interval, value[0])
    max_mean_serial_interval = max(max_mean_serial_interval, value[0])

print('Mean serial interval range: ', [min_mean_serial_interval, max_mean_serial_interval])
print('Serial interval CI range: ', [min_serial_interval, max_serial_interval])

## 1.1 Lode main simulation restuls

In [40]:
code_path = Path('../') 
synthetic_data_path = code_path / 'synthetic_data_results_taiwan_first_outbreak'
taiwan_data_path = code_path / 'data'/'structured_course_of_disease_data'

In [ ]:
demographic_data_list, social_data_list, course_of_disease_data_list, contact_data_list, case_edge_list_list = \
    load_synthetic_data(synthetic_data_path, return_len=100, memory_limit=1e9)


In [ ]:
total_sim_individuals = 0
for i in range(len(demographic_data_list)):
    total_sim_individuals += len(demographic_data_list[i])

print('Total number of simulated individuals: ', total_sim_individuals)

In [ ]:
generation_time_array = np.array([])
serial_interval_array = np.array([])
for i in range(len(course_of_disease_data_list)):
    generation_time_array = np.append(generation_time_array, generate_generation_time(course_of_disease_data_list[i], case_edge_list_list[i]))
    serial_interval_array = np.append(serial_interval_array, generate_generation_time(course_of_disease_data_list[i], case_edge_list_list[i]))

generation_time_array = generation_time_array[~np.isnan(generation_time_array)]
print(len(generation_time_array))

serial_interval_array = serial_interval_array[~np.isnan(serial_interval_array)]
print(len(serial_interval_array))

In [44]:
demographic_data_all = np.concatenate(demographic_data_list, axis=0)
social_data_all = np.concatenate(social_data_list, axis=0)
course_of_disease_data_all = np.concatenate(course_of_disease_data_list, axis=0)
contact_data_all = np.concatenate(contact_data_list, axis=0)

## 1.2 Latent period & incubation period

In [ ]:
fig = plt.figure(figsize=(16, 16))

In [46]:
# Time from infection to symptom-onset
incubation_period = np.array([])
latent_period = np.array([])
for i in range(len(course_of_disease_data_all)):
    incubation_period = np.append(
        incubation_period, course_of_disease_data_all[i]['incubation_period'])
    latent_period = np.append(
        latent_period, course_of_disease_data_all[i]['latent_period'])

In [ ]:
ax1 = fig.add_subplot(3, 3, 1)
ax1.hist(latent_period, bins=np.arange(-0.5, int(np.nanmax(latent_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0], label='Simulation')
ax1.set_xlabel('Latent period (days)')
ax1.set_xlim(1, 50)
ax1.set_ylabel('Frequency')

mean_latent_period = np.nanmean(latent_period)
print(f'Mean latent period: {mean_latent_period:.2f} days')
ax1.axvline(mean_latent_period, color='k', linestyle='dashed', linewidth=2, label='Simulation mean')

ax1.axvspan(min_mean_latent_period, max_mean_latent_period, alpha=0.3, color=current_palette[3], label='Reported mean')
ax1.axvspan(min_latent_period, max_latent_period, alpha=0.3, color=current_palette[1], label='Reported 95% CI')

ax1.legend()

In [ ]:
ax2 = fig.add_subplot(3, 3, 2)
ax2.hist(incubation_period, bins=np.arange(-0.5, int(np.nanmax(incubation_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax2.set_xlabel('Incubation period (days)')
ax2.set_xlim(0, 50)

mean_incubation_period = np.nanmean(incubation_period)
print(f'Mean incubation period: {mean_incubation_period:.2f} days')
ax2.axvline(mean_incubation_period, color='k', linestyle='dashed', linewidth=2)

ax2.axvspan(min_incubation_period, max_incubation_period, alpha=0.3, color=current_palette[1])
ax2.axvspan(min_mean_incubation_period, max_mean_incubation_period, alpha=0.3, color=current_palette[3])

## 1.3 Infectious period

In [49]:
infectious_period = np.array([])
for i in range(len(course_of_disease_data_all)):
    infectious_period = np.append(
        infectious_period, course_of_disease_data_all[i]['infectious_period'])

In [ ]:
print('CovSyn infectious period: ', np.nanmean(infectious_period))

In [ ]:
ax3 = fig.add_subplot(3, 3, 3)
ax3.hist(infectious_period, bins=np.arange(-0.5, int(np.nanmax(infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax3.set_xlabel('Infectious period (days)')
ax3.set_xlim(0, 50)

mean_infectious_period = np.nanmean(infectious_period)
ax3.axvline(mean_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])
print("Mean infectious period range: ", [min_tmp, max_tmp])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])
print("Infectious period Ci range: ", [min_tmp, max_tmp])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])


### 1.3.1 Infectious period for asymptomatic cases

In [52]:
asymptomatic_infectious_period = np.array([])
pre_symptomatic_infectious_period = np.array([])
post_symptomatic_infectious_period = np.array([])
for i, tmp in enumerate(course_of_disease_data_all):
    if np.isnan(course_of_disease_data_all[i]['incubation_period']):
        asymptomatic_infectious_period = np.append(
            asymptomatic_infectious_period, tmp['infectious_period'])
    else:
        pre_symptomatic_infectious_period = np.append(
            pre_symptomatic_infectious_period, tmp['incubation_period']-tmp['latent_period'])
        post_symptomatic_infectious_period = np.append(
            post_symptomatic_infectious_period, tmp['infectious_period']-(tmp['incubation_period']-tmp['latent_period']))

In [ ]:
ax4 = fig.add_subplot(3, 3, 4)
ax4.hist(asymptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(asymptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax4.set_xlabel('Asymptomatic cases infectious period (days)')
ax4.set_xlim(0, 50)
ax4.set_ylabel('Frequency')

mean_asymptomatic_infectious_period = np.nanmean(asymptomatic_infectious_period)
ax4.axvline(mean_asymptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)


min_tmp = 1000
max_tmp = 0
for key, value in report_asymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax4.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_asymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax4.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])


### 1.3.2 Infectious period for presymptomatic cases

In [ ]:
ax5 = fig.add_subplot(3, 3, 5)
ax5.hist(pre_symptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(pre_symptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax5.set_xlabel('Pre-symptomatic cases infectious period (days)')
ax5.set_xlim(0, 50)

mean_pre_symptomatic_infectious_period = np.nanmean(pre_symptomatic_infectious_period)
ax5.axvline(mean_pre_symptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_presymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax5.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_presymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax5.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])



### 1.3.3 Infectious period for postsymptomatic cases

In [ ]:
ax6 = fig.add_subplot(3, 3, 6)
ax6.hist(post_symptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(post_symptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax6.set_xlabel('Post-symptomatic cases infectious period (days)')
ax6.set_xlim(0, 50)

mean_post_symptomatic_infectious_period = np.nanmean(post_symptomatic_infectious_period)
ax6.axvline(mean_post_symptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_postsymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax6.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_postsymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax6.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

## 1.4 Generation time

In [ ]:
ax7 = fig.add_subplot(3, 3, 7)
ax7.hist(generation_time_array, bins=np.arange(-0.5, int(np.nanmax(generation_time_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax7.set_xlabel('Generation time (days)')
ax7.set_xlim(0, 50)
ax7.set_ylabel('Frequency')

mean_generation_time_array = np.nanmean(generation_time_array)
print(f'Mean generation time: {mean_generation_time_array:.2f} days')
ax7.axvline(mean_generation_time_array, color='k', linestyle='dashed', linewidth=2)

ax7.axvspan(min_generation_time, max_generation_time, alpha=0.3, color=current_palette[1])
ax7.axvspan(min_mean_generation_time, max_mean_generation_time, alpha=0.3, color=current_palette[3])

## 1.5 Serial interval

In [ ]:
ax8 = fig.add_subplot(3, 3, 8)
ax8.hist(serial_interval_array, bins=np.arange(int(np.nanmin(serial_interval_array)), int(np.nanmax(serial_interval_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax8.set_xlabel('Serial interval (days)')
ax8.set_xlim(int(np.nanmin(serial_interval_array)), 50)

mean_serial_interval_array = np.nanmean(serial_interval_array)
print(f'Mean serial interval: {mean_serial_interval_array:.2f} days')
ax8.axvline(mean_serial_interval_array, color='k', linestyle='dashed', linewidth=2)

ax8.axvspan(min_serial_interval, max_serial_interval, alpha=0.3, color=current_palette[1])
ax8.axvspan(min_mean_serial_interval, max_mean_serial_interval, alpha=0.3, color=current_palette[3])

## 1.6 R0

In [58]:
R0_array = np.array([])
for contact_data in contact_data_list:
    R0 = R0_average_effective_contact(contact_data)
    R0_array = np.append(R0_array, R0)

In [ ]:
ax9 = fig.add_subplot(3, 3, 9)
ax9.hist(R0_array, bins=len(np.unique(R0_array)), density=False, color=current_palette[0])
ax9.set_xlabel('$R_0$')
# ax9.set_xlim(0, 50)
ax9.set_xscale('log')

mean_R0_array = np.nanmean(R0_array)
print(f'Mean R_0: {mean_R0_array:.2f}')
ax9.axvline(mean_R0_array, color='k', linestyle='dashed', linewidth=2)

ax9.axvspan(min_R0, max_R0, alpha=0.3, color=current_palette[1])
ax9.axvspan(min_mean_R0, max_mean_R0, alpha=0.3, color=current_palette[3])

In [ ]:
fig
# fig.savefig('RW2024_synthetic_data_statistics.pdf', bbox_inches='tight')

In [ ]:
# Dropout the asymptomatic, pre-symptomatic, and post-symptomatic
fig = plt.figure(figsize=(16, 10))

ax1 = fig.add_subplot(2, 3, 1)
ax1.hist(latent_period, bins=np.arange(-0.5, int(np.nanmax(latent_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0], label='Simulation')
ax1.set_xlabel('Latent period (days)')
ax1.set_xlim(1, 50)
ax1.set_ylabel('Frequency')

mean_latent_period = np.nanmean(latent_period)
ax1.axvline(mean_latent_period, color='k', linestyle='dashed', linewidth=2, label='Simulation mean')

ax1.axvspan(min_mean_latent_period, max_mean_latent_period, alpha=0.3, color=current_palette[3], label='Reported mean')
ax1.axvspan(min_latent_period, max_latent_period, alpha=0.3, color=current_palette[1], label='Reported 95% CI')

ax1.legend()

ax2 = fig.add_subplot(2, 3, 2)
ax2.hist(incubation_period, bins=np.arange(-0.5, int(np.nanmax(incubation_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax2.set_xlabel('Incubation period (days)')
ax2.set_xlim(0, 50)

mean_incubation_period = np.nanmean(incubation_period)
ax2.axvline(mean_incubation_period, color='k', linestyle='dashed', linewidth=2)

ax2.axvspan(min_incubation_period, max_incubation_period, alpha=0.3, color=current_palette[1])
ax2.axvspan(min_mean_incubation_period, max_mean_incubation_period, alpha=0.3, color=current_palette[3])

ax3 = fig.add_subplot(2, 3, 3)
ax3.hist(infectious_period, bins=np.arange(-0.5, int(np.nanmax(infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax3.set_xlabel('Infectious period (days)')
ax3.set_xlim(0, 50)

mean_infectious_period = np.nanmean(infectious_period)
ax3.axvline(mean_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

ax7 = fig.add_subplot(2, 3, 4)
ax7.hist(generation_time_array, bins=np.arange(-0.5, int(np.nanmax(generation_time_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax7.set_xlabel('Generation time (days)')
ax7.set_xlim(0, 50)
ax7.set_ylabel('Frequency')

mean_generation_time_array = np.nanmean(generation_time_array)
ax7.axvline(mean_generation_time_array, color='k', linestyle='dashed', linewidth=2)

ax7.axvspan(min_generation_time, max_generation_time, alpha=0.3, color=current_palette[1])
ax7.axvspan(min_mean_generation_time, max_mean_generation_time, alpha=0.3, color=current_palette[3])

ax8 = fig.add_subplot(2, 3, 5)
ax8.hist(serial_interval_array, bins=np.arange(int(np.nanmin(serial_interval_array)), int(np.nanmax(serial_interval_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax8.set_xlabel('Serial interval (days)')
ax8.set_xlim(int(np.nanmin(serial_interval_array)), 50)

mean_serial_interval_array = np.nanmean(serial_interval_array)
ax8.axvline(mean_serial_interval_array, color='k', linestyle='dashed', linewidth=2)

ax8.axvspan(min_serial_interval, max_serial_interval, alpha=0.3, color=current_palette[1])
ax8.axvspan(min_mean_serial_interval, max_mean_serial_interval, alpha=0.3, color=current_palette[3])

ax9 = fig.add_subplot(2, 3, 6)
ax9.hist(R0_array, bins=len(np.unique(R0_array)), density=False, color=current_palette[0])
ax9.set_xlabel('$R_0$')
# ax9.set_xlim(0, 50)
ax9.set_xscale('log')

mean_R0_array = np.nanmean(R0_array)
ax9.axvline(mean_R0_array, color='k', linestyle='dashed', linewidth=2)

ax9.axvspan(min_R0, max_R0, alpha=0.3, color=current_palette[1])
ax9.axvspan(min_mean_R0, max_mean_R0, alpha=0.3, color=current_palette[3])

# fig.savefig('RW2024_synthetic_data_statistics_small.pdf', bbox_inches='tight')

# Add the symptomatic infectious period distribution

In [ ]:
# Calculate symptomatic infectious period
symptomatic_infectious_period = np.array([])
for i, tmp in enumerate(course_of_disease_data_all):
    if not np.isnan(tmp['incubation_period']):
        symptomatic_infectious_period = np.append(
            symptomatic_infectious_period, tmp['infectious_period'])

fig = plt.figure(figsize=(14, 16))

ax1 = fig.add_subplot(4, 3, 1)
ax1.hist(latent_period, bins=np.arange(-0.5, int(np.nanmax(latent_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0], label='Simulation')
ax1.set_xlabel('Latent period (days)')
ax1.set_xlim(1, 50)
ax1.set_ylabel('Frequency')

mean_latent_period = np.nanmean(latent_period)
print(f'Mean latent period: {mean_latent_period:.2f} days')
ax1.axvline(mean_latent_period, color='k', linestyle='dashed', linewidth=2, label='Simulation mean')

ax1.axvspan(min_mean_latent_period, max_mean_latent_period, alpha=0.3, color=current_palette[3], label='Reported mean')
ax1.axvspan(min_latent_period, max_latent_period, alpha=0.3, color=current_palette[1], label='Reported 95% CI')

ax1.legend()

ax2 = fig.add_subplot(4, 3, 2)
ax2.hist(incubation_period, bins=np.arange(-0.5, int(np.nanmax(incubation_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax2.set_xlabel('Incubation period (days)')
ax2.set_xlim(0, 50)

mean_incubation_period = np.nanmean(incubation_period)
print(f'Mean incubation period: {mean_incubation_period:.2f} days')
ax2.axvline(mean_incubation_period, color='k', linestyle='dashed', linewidth=2)

ax2.axvspan(min_incubation_period, max_incubation_period, alpha=0.3, color=current_palette[1])
ax2.axvspan(min_mean_incubation_period, max_mean_incubation_period, alpha=0.3, color=current_palette[3])

ax3 = fig.add_subplot(4, 3, 3)
ax3.hist(infectious_period, bins=np.arange(-0.5, int(np.nanmax(infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax3.set_xlabel('Infectious period (days)')
ax3.set_xlim(0, 50)

mean_infectious_period = np.nanmean(infectious_period)
ax3.axvline(mean_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])
print("Mean infectious period range: ", [min_tmp, max_tmp])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])
print("Infectious period CI range: ", [min_tmp, max_tmp])

ax3.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

ax4 = fig.add_subplot(4, 3, 4)
ax4.hist(asymptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(asymptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax4.set_xlabel('Asymptomatic cases infectious period (days)')
ax4.set_xlim(0, 50)
ax4.set_ylabel('Frequency')

mean_asymptomatic_infectious_period = np.nanmean(asymptomatic_infectious_period)
ax4.axvline(mean_asymptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_asymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax4.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_asymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax4.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

ax5 = fig.add_subplot(4, 3, 5)
ax5.hist(symptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(symptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax5.set_xlabel('Symptomatic cases infectious period (days)')
ax5.set_xlim(0, 50)

mean_symptomatic_infectious_period = np.nanmean(symptomatic_infectious_period)
print(f'Mean symptomatic infectious period: {mean_symptomatic_infectious_period:.2f} days')
ax5.axvline(mean_symptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])
ax5.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])
ax5.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

ax6 = fig.add_subplot(4, 3, 6)
ax6.hist(pre_symptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(pre_symptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax6.set_xlabel('Pre-symptomatic cases infectious period (days)')
ax6.set_xlim(0, 50)

mean_pre_symptomatic_infectious_period = np.nanmean(pre_symptomatic_infectious_period)
ax6.axvline(mean_pre_symptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_presymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax6.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_presymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax6.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

ax7 = fig.add_subplot(4, 3, 7)
ax7.hist(post_symptomatic_infectious_period, bins=np.arange(-0.5, int(np.nanmax(post_symptomatic_infectious_period))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax7.set_xlabel('Post-symptomatic cases infectious period (days)')
ax7.set_xlim(0, 50)
ax7.set_ylabel('Frequency')

mean_post_symptomatic_infectious_period = np.nanmean(post_symptomatic_infectious_period)
ax7.axvline(mean_post_symptomatic_infectious_period, color='k', linestyle='dashed', linewidth=2)

min_tmp = 1000
max_tmp = 0
for key, value in report_postsymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value)])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value)])

ax7.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[1])

min_tmp = 1000
max_tmp = 0
for key, value in report_postsymptomatic_infectious_period.items():
    min_tmp = np.nanmin([min_tmp, np.nanmin(value[0])])
    max_tmp = np.nanmax([max_tmp, np.nanmax(value[0])])

ax7.axvspan(min_tmp, max_tmp, alpha=0.3, color=current_palette[3])

ax8 = fig.add_subplot(4, 3, 8)
ax8.hist(generation_time_array, bins=np.arange(-0.5, int(np.nanmax(generation_time_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax8.set_xlabel('Generation time (days)')
ax8.set_xlim(0, 50)

mean_generation_time_array = np.nanmean(generation_time_array)
print(f'Mean generation time: {mean_generation_time_array:.2f} days')
ax8.axvline(mean_generation_time_array, color='k', linestyle='dashed', linewidth=2)

ax8.axvspan(min_generation_time, max_generation_time, alpha=0.3, color=current_palette[1])
ax8.axvspan(min_mean_generation_time, max_mean_generation_time, alpha=0.3, color=current_palette[3])

ax9 = fig.add_subplot(4, 3, 9)
ax9.hist(serial_interval_array, bins=np.arange(int(np.nanmin(serial_interval_array)), int(np.nanmax(serial_interval_array))+1.5, 1), density=False, rwidth=0.8, color=current_palette[0])
ax9.set_xlabel('Serial interval (days)')
ax9.set_xlim(int(np.nanmin(serial_interval_array)), 50)

mean_serial_interval_array = np.nanmean(serial_interval_array)
print(f'Mean serial interval: {mean_serial_interval_array:.2f} days')
ax9.axvline(mean_serial_interval_array, color='k', linestyle='dashed', linewidth=2)

ax9.axvspan(min_serial_interval, max_serial_interval, alpha=0.3, color=current_palette[1])
ax9.axvspan(min_mean_serial_interval, max_mean_serial_interval, alpha=0.3, color=current_palette[3])

ax10 = fig.add_subplot(4, 3, 10)
ax10.hist(R0_array, bins=len(np.unique(R0_array)), density=False, color=current_palette[0])
ax10.set_xlabel('$R_0$')
ax10.set_xscale('log')
ax10.set_ylabel('Frequency')

mean_R0_array = np.nanmean(R0_array)
print(f'Mean R_0: {mean_R0_array:.2f}')
ax10.axvline(mean_R0_array, color='k', linestyle='dashed', linewidth=2)

ax10.axvspan(min_R0, max_R0, alpha=0.3, color=current_palette[1])
ax10.axvspan(min_mean_R0, max_mean_R0, alpha=0.3, color=current_palette[3])

plt.tight_layout()
plt.subplots_adjust(hspace=0.3)
# fig
fig.savefig('RW2024_synthetic_data_statistics.pdf', bbox_inches='tight')

In [ ]:
# Latent = incubation = isolation
equal_counts = 0
for i in range(len(course_of_disease_data_all)):
    if (course_of_disease_data_all[i]['latent_period'] == course_of_disease_data_all[i]['incubation_period']) & \
            (course_of_disease_data_all[i]['incubation_period'] == course_of_disease_data_all[i]['monitor_isolation_period']):
        equal_counts += 1

plt.bar(range(2), [equal_counts, len(
    course_of_disease_data_all)-equal_counts])
plt.xticks(range(2), ['Equal', 'Not equal'])


In [ ]:
# Check how early of each effective contacts could be.
time_from_infectious_to_effective_contact = []
for i in range(len(course_of_disease_data_list)):
    courst_tmp = course_of_disease_data_list[i]
    case_edge_tmp = case_edge_list_list[i]
    for j in range(len(case_edge_tmp)):
        if type(case_edge_tmp[j][0]) == np.str_:
            if case_edge_tmp[j][0] != 'nan':
                source = int(case_edge_tmp[j][0])-1
                source_infection_day = courst_tmp[source]['infection_day']
                source_latent_period = courst_tmp[source]['latent_period']
                target = int(case_edge_tmp[j][1])-1
                target_infection_day = courst_tmp[target]['infection_day']
                time_from_infectious_to_effective_contact.append(
                    target_infection_day - source_infection_day - source_latent_period)
        else:
            if not np.isnan(case_edge_tmp[j][0]):
                source = int(case_edge_tmp[j][0])-1
                source_infection_day = courst_tmp[source].infection_day
                source_latent_period = courst_tmp[source].latent_period
                target = int(case_edge_tmp[j][1])-1
                target_infection_day = courst_tmp[target].infection_day
                time_from_infectious_to_effective_contact.append(
                    target_infection_day - source_infection_day - source_latent_period)

plt.hist(time_from_infectious_to_effective_contact, bins=range(0, max(time_from_infectious_to_effective_contact)+1))
plt.xlabel('Time from source infectious day to effective contact day')
plt.ylabel('Counts')
print(f'Mean time from source infectious day to effective contact day: {np.mean(time_from_infectious_to_effective_contact):.2f}')

In [ ]:
time_from_infectious_to_first_contact = []
for i in range(len(course_of_disease_data_list)):
    contact_tmp = contact_data_list[i]
    courst_tmp = course_of_disease_data_list[i]
    for j in range(len(contact_tmp)):
        household_effective_contacts = contact_tmp[j]['household_effective_contacts']
        for k, household_effective_contact in enumerate(household_effective_contacts):
            if household_effective_contact == 1:
                first_contact_day = np.where(contact_tmp[j]['household_contacts_matrix'][k])[0][0]
                first_contact_day = first_contact_day - courst_tmp[j]['latent_period']
                time_from_infectious_to_first_contact.append(first_contact_day)

        school_effective_contacts = contact_tmp[j]['school_effective_contacts']
        for k, school_class_effective_contact in enumerate(school_effective_contacts):
            if school_class_effective_contact == 1:
                first_contact_day = np.where(contact_tmp[j]['school_class_contacts_matrix'][k])[0][0]
                first_contact_day = first_contact_day - courst_tmp[j]['latent_period']
                time_from_infectious_to_first_contact.append(first_contact_day)

        workplace_effective_contacts = contact_tmp[j]['workplace_effective_contacts']
        for k, workplace_effective_contact in enumerate(workplace_effective_contacts):
            if workplace_effective_contact == 1:
                first_contact_day = np.where(contact_tmp[j]['workplace_contacts_matrix'][k])[0][0]
                first_contact_day = first_contact_day - courst_tmp[j]['latent_period']
                time_from_infectious_to_first_contact.append(first_contact_day)

        health_care_effective_contacts = contact_tmp[j]['health_care_effective_contacts']
        for k, health_care_effective_contact in enumerate(health_care_effective_contacts):
            if health_care_effective_contact == 1:
                first_contact_day = np.where(contact_tmp[j]['health_care_contacts_matrix'][k])[0][0]
                first_contact_day = first_contact_day - courst_tmp[j]['latent_period']
                time_from_infectious_to_first_contact.append(first_contact_day)

        municipality_effective_contacts = contact_tmp[j]['municipality_effective_contacts']
        for k, municipality_effective_contact in enumerate(municipality_effective_contacts):
            if municipality_effective_contact == 1:
                first_contact_day = np.where(contact_tmp[j]['municipality_contacts_matrix'][k])[0][0]
                first_contact_day = first_contact_day - courst_tmp[j]['latent_period']
                time_from_infectious_to_first_contact.append(first_contact_day)
            
plt.hist(time_from_infectious_to_first_contact, bins=range(0, max(time_from_infectious_to_first_contact)+1))
plt.xlabel('Time from source infectious day to first contact day of the effective contact')
plt.ylabel('Counts')
print(f'Mean time from source infectious day to first contact day: {np.mean(time_from_infectious_to_first_contact):.2f}')

# 2. Check demographic data

In [ ]:
age = np.ones(len(demographic_data_all))
for i in range(len(age)):
    age[i] = demographic_data_all[i]['age']

plt.hist(age, bins=range(0,int(np.nanmax(age)+1)))
plt.xlabel('Age')
plt.ylabel('Counts')   
plt.xlim(0, 100)

In [ ]:
# This code is to check if index for each data type is correct
# Plot age vs school
for i in range(len(demographic_data_all)):
    age = demographic_data_all[i]['age']
    school_size = social_data_all[i]['school_class_size']

    plt.plot(age, school_size, 'k*')

plt.xlabel('Age')
plt.ylabel('School size')

# Plot age vs workplace
for i in range(len(demographic_data_all)):
    age = demographic_data_all[i]['age']
    workplace_size = social_data_all[i]['work_group_size']

    plt.plot(age, workplace_size, 'k*')

plt.xlabel('Age')
plt.ylabel('workplace size')


In [ ]:
household_effective_contact_array = np.array([])
school_effective_contact_array = np.array([])
workplace_effective_contact_array = np.array([])
health_care_effective_contact_array = np.array([])
municipality_effective_contact_array = np.array([])
for i in range(len(contact_data_all)):
    if np.nansum(contact_data_all[i]['household_effective_contacts_infection_time']) > 0:
        household_effective_contact_array = np.append(household_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['household_effective_contacts_infection_time']))))
    else:
        household_effective_contact_array = np.append(
            household_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['school_effective_contacts_infection_time']) > 0:
        school_effective_contact_array = np.append(school_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['school_effective_contacts_infection_time']))))
    else:
        school_effective_contact_array = np.append(
            school_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['workplace_effective_contacts_infection_time']) > 0:
        workplace_effective_contact_array = np.append(workplace_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['workplace_effective_contacts_infection_time']))))
    else:
        workplace_effective_contact_array = np.append(
            workplace_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['health_care_effective_contacts_infection_time']) > 0:
        health_care_effective_contact_array = np.append(health_care_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['health_care_effective_contacts_infection_time']))))
    else:
        health_care_effective_contact_array = np.append(
            health_care_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['municipality_effective_contacts_infection_time']) > 0:
        municipality_effective_contact_array = np.append(municipality_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['municipality_effective_contacts_infection_time']))))
    else:
        municipality_effective_contact_array = np.append(
            municipality_effective_contact_array, 0)

all_effective_contact_array = household_effective_contact_array + school_effective_contact_array + \
    workplace_effective_contact_array + health_care_effective_contact_array + \
    municipality_effective_contact_array

plt.hist(all_effective_contact_array, bins=np.arange(-0.5, int(np.nanmax(all_effective_contact_array))+1.5, 1), rwidth=0.8)
plt.xlabel('Effective contacts')
plt.ylabel('Counts')
plt.xlim(np.nanmin(all_effective_contact_array)-0.5, np.nanmax(all_effective_contact_array)+0.5)
# plt.yscale('log')
print('Max number of effective contacts: ', np.nanmax(all_effective_contact_array))